# Import packages

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

# 1. Load dataset

In [3]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


# 2. Preprocessing

## 1) Imputing

In [4]:
train_data = impute_data(train_data)

Number of Nan values: 0


## 2) Feature engineering

In [5]:
train_data['Wspd_cos'] = train_data['Wspd']*np.cos(train_data['Wdir']/180*np.pi)

alpha = 20
train_data['TSR1'] = 1/np.tan(np.radians(train_data['Pab1']+alpha))
train_data['TSR2'] = 1 / np.tan(np.radians(train_data['Pab2'] + alpha))
train_data['TSR3'] = 1 / np.tan(np.radians(train_data['Pab3'] + alpha))    

train_data['Bspd1'] = train_data['TSR1'] * train_data['Wspd_cos']
train_data['Bspd2'] = train_data['TSR2'] * train_data['Wspd_cos']
train_data['Bspd3'] = train_data['TSR3'] * train_data['Wspd_cos']

train_data['rpm'] = (train_data['Bspd1'] + train_data['Bspd2'] + train_data['Bspd3']) / 3

## 3) Convert dtype of `Tmstamp`

In [6]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)

## 4) Split dataset

In [7]:
SEQ_LEN = 5
(train_x, train_y), (val_x, val_y), (test_x, test_y) = make_train_val_test_data(train_data, SEQ_LEN)

100%|██████████| 134/134 [00:26<00:00,  5.11it/s]


Train data(X, y): (24522, 720, 19) (24522, 288)
Validation data(X, y): (134, 720, 19) (134, 288)
Test data(X, y): (134, 720, 19) (0,)


# 3. Modeling

In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-07-12 05:38:37.041591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 05:38:37.331773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 05:38:37.332109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [9]:
model = Sequential([
    GRU(256, input_shape=train_x[0].shape),
    Dense(516, activation='relu'),
    Dense(288, activation='relu')
])
optimizer = tf.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

2022-07-12 05:38:37.851813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-12 05:38:37.859497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 05:38:37.859930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 05:38:37.860208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# 4. Training

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1000, batch_size=128, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]);

Epoch 1/1000


2022-07-12 05:38:47.500289: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2022-07-12 05:38:51.572983: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


192/192 [==============================] - 17s 38ms/step - loss: 186806.4219 - mae: 327.3302 - val_loss: 305753.7812 - val_mae: 470.8974
Epoch 2/1000
192/192 [==============================] - 7s 37ms/step - loss: 161811.7031 - mae: 311.8778 - val_loss: 248179.7188 - val_mae: 418.9817
Epoch 3/1000
192/192 [==============================] - 7s 37ms/step - loss: 152559.5625 - mae: 300.2212 - val_loss: 250522.8281 - val_mae: 424.2743
Epoch 4/1000
192/192 [==============================] - 7s 37ms/step - loss: 145624.3125 - mae: 290.9237 - val_loss: 256926.0000 - val_mae: 443.5643
Epoch 5/1000
192/192 [==============================] - 7s 37ms/step - loss: 139491.1406 - mae: 282.6794 - val_loss: 230259.2500 - val_mae: 411.8481
Epoch 6/1000
192/192 [==============================] - 7s 37ms/step - loss: 133880.4375 - mae: 275.3751 - val_loss: 226343.1406 - val_mae: 395.2729
Epoch 7/1000
192/192 [==============================] - 7s 37ms/step - loss: 128325.2344 - mae: 268.3458 - val_loss: 2

# 6. Generate submission

In [11]:
sample_submission['Patv'] = model.predict(test_x).reshape(-1)
sample_submission.to_csv(join(PATH.output, "proposed6.csv"), index=False)

5/5 [==============================] - 0s 9ms/step
